# 第 7 章 数据清洗和准备

In [8]:
import pandas as pd
import numpy as np

## 7.1 处理缺失数据

pandas 的目标之一就是尽量 轻松地处理缺失数据。

对于数值数据，pandas 使用浮点值 NaN(Not a Number)表示缺失数 据。我们称其为哨兵值，可以方便的检测出来:

In [9]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan,'avocado'])

In [10]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [11]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python 内置的 None 值在对象数组中也可以作为 NA:

In [12]:
string_data[0] = None

In [14]:
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [13]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

## 滤除缺失数据

你可以通过 pandas.isnull 或布尔索引的手 工方法，但 dropna 可能会更实用一些。对于一个 Series，dropna 返回一个仅 含非空数据和索引值的 Series:

In [15]:
from numpy import nan as NA

In [16]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [19]:
data.dropna() #dropna 返回一个仅 含非空数据和索引值的 Series:

0    1.0
2    3.5
4    7.0
dtype: float64

In [20]:
#这等价于:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于 DataFrame 对象，事情就有点复杂了。你可能希望丢弃全 NA 或含有 NA 的行或列。dropna 默认丢弃任何含有缺失值的行:

In [41]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [23]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [25]:
cleaned = data.dropna()  #dropna 默认丢弃任何含有缺失值的行
cleaned 

,0,1,2
0,1.0,6.5,3.0


传入 how='all'将只丢弃全为 NA 的那些行:

In [42]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入 axis=1 即可:

In [43]:
data[4] = NA

In [44]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [45]:
data.dropna(axis=1, how='all') #丢弃一列全是NaN的

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除 DataFrame 行的问题涉及时间序列数据。假设你只想留下一部分观 测数据，可以用 thresh 参数实现此目的:

In [49]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,-0.828930,0.391449,0.464204
1,0.434492,-0.050273,0.461938
2,-0.440553,0.538571,-1.895166
3,-1.110627,0.782413,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [51]:
df.iloc[:4, 1] = NA   #直接在原df上修改
df

,0,1,2
0,-0.828930,NaN,0.464204
1,0.434492,NaN,0.461938
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [52]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.828930,NaN,NaN
1,0.434492,NaN,NaN
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [53]:
df.dropna() #带有NA的都去掉

,0,1,2
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [54]:
df.dropna(thresh=2)   #行中带有2个NA的去掉

,0,1,2
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [75]:
df
df.dropna(thresh=2)   #传入thresh=n保留至少有n个非NaN数据的行：

,0,1,2
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


## 填充缺失数据

对于大多数情况而言，fillna 方法是最主要的 函数。通过一个常数调用 fillna 就会将缺失值替换为那个常数值:

In [77]:
df.fillna(0)   #fillna 就会将缺失值替换为常数值

,0,1,2
0,-0.828930,0.000000,0.000000
1,0.434492,0.000000,0.000000
2,-0.440553,0.000000,-1.895166
3,-1.110627,0.000000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


若是通过一个字典调用 fillna，就可以实现对不同的列填充不同的值:

In [78]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.828930,0.500000,0.000000
1,0.434492,0.500000,0.000000
2,-0.440553,0.500000,-1.895166
3,-1.110627,0.500000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


fillna 默认会返回新对象，但也可以对现有对象进行就地修改:

In [79]:
_ = df.fillna(0, inplace=True)

In [81]:
df

,0,1,2
0,-0.828930,0.000000,0.000000
1,0.434492,0.000000,0.000000
2,-0.440553,0.000000,-1.895166
3,-1.110627,0.000000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


对 reindexing 有效的那些插值方法也可用于 fillna:

In [83]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,-1.119467,1.163443
3,0.952511,-0.316789,-0.679762
4,1.760643,0.938081,-1.088436
5,-1.953855,-0.214966,-0.341011


In [85]:
df.iloc[2:, 1] = NA
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,NaN,1.163443
3,0.952511,NaN,-0.679762
4,1.760643,NaN,-1.088436
5,-1.953855,NaN,-0.341011


In [87]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,NaN,1.163443
3,0.952511,NaN,-0.679762
4,1.760643,NaN,NaN
5,-1.953855,NaN,NaN


In [88]:
df.fillna(method='ffill')  ## todo

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,0.661740,1.163443
3,0.952511,0.661740,-0.679762
4,1.760643,0.661740,-0.679762
5,-1.953855,0.661740,-0.679762


In [89]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,0.661740,1.163443
3,0.952511,0.661740,-0.679762
4,1.760643,NaN,-0.679762
5,-1.953855,NaN,-0.679762


只要有些创新，你就可以利用 fillna 实现许多别的功能。比如说，你可以传入 Series 的平均值或中位数:

In [91]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [92]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 数据转换

## 移除重复数据

DataFrame 中出现重复行有多种原因。下面就是一个例子:

In [93]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [94]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame 的 duplicated 方法返回一个布尔型 Series，表示各行是否是重复行 (前面出现过的行):

In [95]:
data.duplicated()  #判断各行是否重复

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有一个与此相关的 drop_duplicates 方法，它会返回一个 DataFrame，重复 的数组会标为 False:

In [96]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我 们还有一列值，且只希望根据 k1 列过滤重复项:

In [98]:
data['v1'] = range(7)

In [99]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [100]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated 和 drop_duplicates 默认保留的是第一个出现的值组合。传入 keep='last'则保留最后一个:

In [102]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 利用函数或映射进行数据转换

## 替换值

## 重命名轴索引

## 离散化和面元划分

## 检测和过滤异常值

## 排列和随机采样

## 计算指标/哑变量

## 7.3 字符串操作

## 字符串对象方法

## 正则表达式

## pandas 的矢量化字符串函数

## 7.4 总结